In [47]:
import pandas as pd

In [51]:
ped = pd.read_csv("köln-schildergasse (west)-20180430-20200131-hour.csv", sep = ";")
holi = pd.read_csv("Feiertage_2018.01.01_2020.01.31.csv", sep = ";")
print(ped.head(3))
print(holi.head(3))



                     location        time of measurement  weekday  \
0  Schildergasse (West), Köln  2018-05-01 00:00:00 +0200  Tuesday   
1  Schildergasse (West), Köln  2018-05-01 01:00:00 +0200  Tuesday   
2  Schildergasse (West), Köln  2018-05-01 02:00:00 +0200  Tuesday   

   pedestrians count  temperature in ºc    weather condition  incidents  
0                  0                8.0  partly-cloudy-night        NaN  
1                  0                7.0  partly-cloudy-night        NaN  
2                146                7.0  partly-cloudy-night        NaN  
         date        name
0  2018-01-01     Neujahr
1  2018-03-30  Karfreitag
2  2018-04-01      Ostern


In [52]:
#Splitting up the Time of measurement into date and time
ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)
#Drpo Time of measuremnt, because it is split now
ped = ped.drop(["time of measurement"], axis=1)
#Drop incidents, if only nan
if ped["incidents"].unique() == "nan":
    ped = ped.drop(["incidents"], axis=1)

C:\Users\Biebert\AppData\Local\Temp\ipykernel_17388\551244843.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)
C:\Users\Biebert\AppData\Local\Temp\ipykernel_17388\551244843.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if ped["incidents"].unique() == "nan":


In [53]:
# restructure by date
ped = ped[ ["date", "time"] + [ col for col in ped.columns if col != "date" and col!= "time"] ]
#change name
holi = holi.rename(columns={"name": "holiday"})
#merge on date
ped = pd.merge(ped, holi, how = "left")

In [54]:
# clear time from the minute variation
ped["time"] = ped["time"].str.slice_replace(8, 16)
ped

,date,time,location,weekday,pedestrians count,temperature in ºc,weather condition,incidents,holiday
0,2018-05-01,00:00:00,"Schildergasse (West), Köln",Tuesday,0,8.0,partly-cloudy-night,NaN,Tag der Arbeit
1,2018-05-01,01:00:00,"Schildergasse (West), Köln",Tuesday,0,7.0,partly-cloudy-night,NaN,Tag der Arbeit
2,2018-05-01,02:00:00,"Schildergasse (West), Köln",Tuesday,146,7.0,partly-cloudy-night,NaN,Tag der Arbeit
3,2018-05-01,03:00:00,"Schildergasse (West), Köln",Tuesday,125,7.0,partly-cloudy-night,NaN,Tag der Arbeit
4,2018-05-01,04:00:00,"Schildergasse (West), Köln",Tuesday,84,6.0,partly-cloudy-night,NaN,Tag der Arbeit
...,...,...,...,...,...,...,...,...,...
15380,2020-01-31,19:00:00,"Schildergasse (West), Köln",Friday,5422,11.0,partly-cloudy-night,NaN,NaN
15381,2020-01-31,20:00:00,"Schildergasse (West), Köln",Friday,2691,11.0,partly-cloudy-night,NaN,NaN
15382,2020-01-31,21:00:00,"Schildergasse (West), Köln",Friday,1329,10.0,partly-cloudy-night,NaN,NaN
15383,2020-01-31,22:00:00,"Schildergasse (West), Köln",Friday,915,11.0,cloudy,NaN,NaN
